In [7]:
# this class extract bvh file from a directory
class extract_bvh:
    
    def __init__(self, nb_frames, nb_joints):
        # 3 coordinates for each joints (in our case)
        self.nb_joints = nb_joints
        self.nb_frames = nb_frames
        
    def __open_bvh(self, path):
        # extract bvh file if it exist
        mvt = np.array([])
        with open(path) as f:
            mvt = np.array(Bvh(f.read()).frames, dtype="float").T[:self.nb_joints]
        return mvt
    
    def __normalize(self, m):
        # return the normalized matrix using the init parameters
        m_n = np.zeros((self.nb_joints, self.nb_frames))
        dim_m = m.shape[1]
        #print(m.shape)
        i = 0
        while i < m.shape[0]:
            f = interp1d(np.arange(dim_m), m[i,:], fill_value='extrapolate')
            m_n[i,:] = np.array(f(np.linspace(0, dim_m, self.nb_frames)))
            i += 1
            
        return m_n
    
    def extract_one(self, path):
        # get the normalized matric of the bvh file of the path
        return self.__normalize(self.__open_bvh(path))
    
    def extract_from_directory(self, path, cat):
        files = os.listdir(path)
        # remove all no-bvh files
        files = self.__remove_no_bvh_file(files, path)
        # recuperation du nombre de fichier pour initialiser la matrice
        nb_files = np.array(files).size
        print("cat :", cat, " nb files :", nb_files)
        # initialisation de la matrice
        # 3 dimensions : categorie, frames, joints
        mvts = np.zeros((nb_files, self.nb_joints, self.nb_frames))
        i = 0
        for f in files:
            path_f = path + f
            mvts[i] = np.array(self.extract_one(path_f))
            i += 1
            
        return np.array([[cat, x] for x in mvts])
    
    def __remove_no_bvh_file(self, file_list, path):
        for f in file_list:
            path_f = path + f
            fileName, fileExtension = os.path.splitext(path_f)
            if fileExtension != '.bvh':
                file_list.remove(f)
        
        return file_list

In [4]:
# retreive data from folder
# one folder per categorie
# use bed
class bvh_extract_data:
    
    data = np.array([])
        
    def __init__(self, p_nb_frames, p_nb_coord, p_path):
        self._nb_frames = p_nb_frames
        self._nb_coord = p_nb_coord
        self._path = p_path
        self.tab_cat = os.listdir(p_path)
        
        self._extract_data()
        
    def _extract_data(self):
        eb = extract_bvh(self._nb_frames, self._nb_coord)
        nb_cat = len(self.tab_cat)
        mvts = np.array([])
        i = 0
        while i < nb_cat:
            folder = self.tab_cat[i]
            mvts_tmp = eb.extract_from_directory(self._path + 
                                                 folder + '/', i)
            #print("size mvts_tmp", mvts_tmp.shape)
            if mvts.size == 0:
                mvts = np.array([x for x in mvts_tmp])
            else:  
                mvts = np.array([x for x in [mvts, mvts_tmp]])
            i+=1
            
            
        self.data = mvts
        
    def create_database(self, train_db, test_db):
        
        x_train = np.array([self.data[0][i][1] for i in range(4)])
        x_train = np.vstack([x_train, [self.data[1][i][1] for i in range(4)]])
        x_test = np.array([self.data[0][i][1] for i in range(4,6)])
        x_test = np.vstack([x_test, [self.data[1][i][1] for i in range(4,6)]])
        
        y_train = np.array([self.data[0][i][0] for i in range(4)])
        y_train = np.vstack([y_train, [self.data[1][i][0] 
                                       for i in range(4)]]).flatten()
        y_test = np.array([self.data[0][i][0] for i in range(4,6)])
        y_test = np.vstack([y_test, [self.data[1][i][0] 
                                     for i in range(4,6)]]).flatten()
        
        res_y_train = np.zeros((8,2))
        i = 0
        for j in y_train:
            res_y_train[i,j] = 1
            i+=1
            
        res_y_test = np.zeros((4,2))
        i = 0
        for j in y_test:
            res_y_test[i,j] = 1
            i+=1
        
        return (x_train, res_y_train, x_test, res_y_test)
    

In [2]:
# récup data from folder with excel
class bvh_data_extract:

    def __init__(self, path, p_nb_frames, p_nb_coord):
        
        self.data = np.array([])
        
        self._nb_frames = p_nb_frames
        self._nb_coord = p_nb_coord
        self._excel_file = path
        
        self.list_cat = np.array([])
        self.list_cat_sel = np.array([])
        self.list_nb_cat_sel = np.array([])
        self.id_cat_sel = np.array([])
        
    def extract_database(self):
        if self._is_exists() == 1:
            self._open_excel()
        else:
            return 0
        
    def _is_exists(self):
        if os.path.exists(self._excel_file) == 0:
            print('The excel files doesn\'t exists')
            return 0
        return 1

    def _name_folder(self, file):
        res = ""
        i=0
        # nom de fichier de la forme dossier_fichier.bvh
        while file[i] != '_':
            res += file[i]
            i += 1
            
        return res
        
    def _open_excel(self):
        bdd = pd.read_excel(self._excel_file)
        # extraction des fichiers
        ext_bvh = extract_bvh(self._nb_frames,self._nb_coord)
        
        # lister tous les mouvements
        self.list_cat = np.unique(bdd['categories'].to_list(), return_counts=True)
        # catégories des mouvements
        nb_cat = self.list_cat[0].shape[0]
        # nb max de mouvement par catégorie
        nb_mvt_max = max(self.list_cat[1])
        
        # init du tableau
        mvts_cmu = np.empty([nb_cat, nb_mvt_max], dtype='S20')
        mvts_bvh = np.zeros((1,2))
        #indice tableau cu data
        i = 0
        # récupération de chaque mouvement
        for cat in self.list_cat[0]:
            
            if cat == 'nan':
                tmp_mvts= np.array([])
            else:
                print("cat utilisee :", cat)
                self.list_cat_sel = np.append(self.list_cat_sel,cat)
                
                tmp_mvts = bdd[bdd['categories'] == cat]

                #indice tableau cmu data, nb cat
                j = 0
                for file in tmp_mvts['MOTION']:
                    folder = self._name_folder(file)

                    path = 'data/cmu/' + folder + '/' + file + '.bvh'
                    #print(path)
                    mvts_cmu[i,j] = path

                    mvts_bvh = np.vstack((mvts_bvh,[i, ext_bvh.extract_one(path)]))

                    j += 1
                print("nb mvt", j)
                self.list_nb_cat_sel = np.append(self.list_nb_cat_sel,j)
                self.id_cat_sel = np.append(self.id_cat_sel,i)
                i += 1
            
        self.data = mvts_bvh[1:]        
    
    # création de x_train, y_train, x_test et y_test
    def create_database(self, train_db, test_db):

        # nombre de categorie
        self.nb_cat = len(self.list_cat_sel)
        print("Nombre de catégorie :", self.nb_cat)
        
        test = np.array([bde.data[bde.data[:,0]==x][test_db[0]:test_db[1]] 
                         for x in self.id_cat_sel])
        train = np.array([bde.data[bde.data[:,0]==x][train_db[0]:train_db[1]] 
                          for x in self.id_cat_sel])
        
        nb_mvt_train = train.shape[0] * train.shape[1]
        nb_mvt_test = test.shape[0] * test.shape[1]

        y_test = np.zeros((nb_mvt_test,self.nb_cat))
        x_test = np.zeros((1,self._nb_coord,self._nb_frames))

        y_train = np.zeros((nb_mvt_train,self.nb_cat))
        x_train = np.zeros((1,self._nb_coord, self._nb_frames))
        
        n_mvt = 0
        for i in test:
            for j in i:
                y_test[n_mvt, j[0]] = 1
                x_test = np.vstack([x_test, [j[1]]])
                n_mvt += 1

        n_mvt = 0
        for i in train:
            for j in i:
                y_train[n_mvt, j[0]] = 1
                x_train = np.vstack([x_train, [j[1]]])
                n_mvt += 1

        x_test = x_test[1:]
        x_train = x_train[1:]

        y_test = y_test
        y_train = y_train
        
        return (x_train, y_train, x_test, y_test)
        